In [7]:
%cd ..

a:\Monthy\5_2023\Crawl_buffett


In [8]:
import PyPDF2
import re
import os
import time
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore", UserWarning)


In [9]:
from get_volume import convert_pdf_to_text


In [10]:
path = 'Data/5486/PDF/2008_Q1_決算短信(2008_7_30).pdf'